# ¡Query Time!

Ahora, vamos a comparar operaciones parecidas con ambas bases de datos.

Comencemos haciendo un pequeño set-up:

- El **firewall**! Este notebook tiene una dirección IP diferente, así que debemos agregarla a las reglas de Firewall **¿Alguien me puede guiar?**
- **Instalar** las **librerías** necesarias
- **Conectarse** a las bases de datos

**ATENCIÓN!** -> Para **MongoDB** necesitaremos habilitar la regla de firewall para acceder a la máquina virtual. Pero para **PostgreSQL**, como es un servicio autogestionado de Google Cloud, necesitaremos ir a la propia DB para habilitar la red: SQL -> dota -> Connections -> Networking -> Add a network

In [ ]:
!curl ipecho.net/plain

In [ ]:
!pip install psycopg2-binary pymongo

### Vamos a conectarnos a nuestra DB Postgre:

Necesitamos la IP!
Vayamos a SQL -> Instances a ver nuestra DB

In [ ]:
dbname='postgres'
ip='34.134.179.7' # Tu IP

print("Conectando a la siguiente base de datos POSTGRE:")
print("dbname={} user=dota password=dota host={}".format(dbname, ip))

In [ ]:
import psycopg2
conn = psycopg2.connect(#Aquí tu código#)
postgres = conn.cursor()

### Conectémonos a nuestra DB Mongo:

En ocasiones, si tenemos algún error con la conexión al server de Mongo, los problemas pueden tener varios orígenes:
- Nuestra IP ha cambiado
- La IP de la máquina virtual ha cambiado
- Algún proceso de Mongo se ha quedado colgado en el servidor y debemos reiniciarlo (´"sudo systemctl restart mongod" en la consola SSH de la máquina)

In [ ]:
from pymongo import MongoClient
from pprint import pprint
client = MongoClient(#Aquí tu código#)
mongodb = client.dota

# Querys sencillas:

Ahora que ya estamos conectados a ambas Bases de Datos, comencemos con algunas consultas sencillas:



### Postgre:

In [ ]:
# Vamos a hacer un select en la tabla heroes para que nos devuelva la información de Tiny
postgres.execute(#Aquí tu código#)
postgres.fetchone()

### Mongo:

In [ ]:
mongodb.heroes.find_one(#Aquí tu código#)

# Querys con **LIKE**

### PostgreSQL

In [ ]:
# Hagamos una consulta que nos devuelva todo aquello de la tabla items donde la descripción blade y más cosas
postgres.execute(#Aquí tu código#)
postgres.fetchone()

### MongoDB

In [ ]:
mongodb.heroes.find_one(#Aquí tu código#)

# Count

# PostgreSQL

In [ ]:
# Vamos a devolver el número de filas que existen en la tabla heroes cuando tipo sea Strength
postgres.execute(#Aquí tu código#)
postgres.fetchone()

### MongoDB

In [ ]:
mongodb.heroes.count_documents(#Aquí tu código#)

# Nuevas Colecciones/Tablas

### PostgreSQL

In [ ]:
try:
  postgres.execute("""
CREATE TABLE quests(
   ID           INT PRIMARY KEY NOT NULL,
   name         CHAR(50) NOT NULL,
   description  CHAR(250) NOT NULL,
   exp          INT NOT NULL
)""")
  conn.commit()
except:
  conn.rollback()
  print("table exists")

postgres.execute("SELECT * FROM quests")
postgres.fetchone()

### MongoDB

In [ ]:
mongodb.quests.insert_one({
    "name": "find sword",
    "description": "Find the hidden sword",
    "exp": 500
    })
mongodb.quests.find_one()

In [ ]:
mongodb.quests.insert_one({
    "result": 500,
    "id": 1
    })

mongodb.quests.find_one({"id": 1})

# Inserción de datos

### PostgreSQL

In [ ]:
# Vamos a insertar: INSERT INTO quests (id, name, description, exp) VALUES (100, 'find sword', 'Find the hidden sword', 500)
try:
  #Aquí tu código#
except:
  conn.rollback()
  print("INSERT ALREADY DONE BY OTHER STUDENT")


postgres.execute("select * from quests")
postgres.fetchone()

### MongoDB

In [ ]:
# Ahora insertemos en Mongo al personaje con nombre: Squall y valor: 1
mongodb.quests.insert_many([
    #Aquí tu código#
])
quests = mongodb.quests.find()
for quest in quests:
  print(quest)

__NOTA:__ Los documentos en MongoDB no tienen restricciones de insercion por primary key (se duplican).

# Joins

### PostgreSQL

In [ ]:
# Vamos a buscar en todas las columnas de las tablas heroes y skills cuando el nombre sea 'Pugna'

try:
  postgres.execute(#Aquí tu código#)
  heroes_skills = postgres.fetchall()
except:
  conn.rollback()
  print("INSERT ALREADY DONE BY OTHER STUDENT")

for hero in heroes_skills:
    pprint(hero)

### MongoDB

In [ ]:
mongodb.heroes.find_one(
    #Aquí tu código#
    )

**No hay que hacer Joins!**

# Modificar tablas - Añadir columnas

### PostgreSQL

In [ ]:
# Vamos a agegar a la tabla heroes la columna 'favorite_items'

try:
  postgres.execute(#Aquí tu código#
                   )
  conn.commit()
except:
  conn.rollback()
  print("ALTER ALREADY DONE BY OTHER STUDENT")

In [ ]:
# Vamos a actualizar el valor de la tabla heroes estableciendo el valor de la columna 'favorite_items'
# a 'Teleport Scroll, magic wand, sword' cuando el name sea 'Tiny'
try:
  postgres.execute(#Aquí tu código#
                   )
  conn.commit()
except:
  conn.rollback()
  print("UPDATE ALREADY DONE BY OTHER STUDENT")

postgres.execute("select * from heroes where name = 'Tiny'")
postgres.fetchone()

### MongoDB

In [ ]:
mongodb.heroes.update_many(
	#Aquí tu código#
)
mongodb.heroes.find_one({"name": "Tiny"})

# Relacionar datos nuevos entre tablas/colecciones

### PostgreSQL

In [ ]:
# Vamos a ponerle al heroe con id 33 el item con id 26
try:
  postgres.execute(#Aquí tu código#
                   )
  conn.commit()
except:
  conn.rollback()
  print("INSERT ALREADY DONE BY OTHER STUDENT")

postgres.execute("select * from heroes_items")
postgres.fetchone()

In [ ]:
postgres.execute("""
    SELECT * FROM heroes
    LEFT JOIN heroes_items ON heroes.id = heroes_items.hero_id
    LEFT JOIN items ON heroes_items.item_id = items.id
    WHERE heroes.id = 33;
""")
postgres.fetchone()

### MongoDB

In [ ]:
# Ahora vamos a agregar al heroe con name 'Visage' el item 'Blades_of_Attack'

mongodb.heroes.update_one(
	{"name": "Visage"},
	{"$push": {"items": mongodb.items.find_one({"name": "Blades_of_Attack"})}}
)
mongodb.heroes.find_one({"name": "Visage"})

# Modificar datos existentes

### PostgreSQL

In [ ]:
# Vamos a actualizar el valor de Sacred_Relic a MEGA SWORD
try:
  postgres.execute(#Aquí tu código#
                   )
  conn.commit()
except:
  conn.rollback()
  print("UPDATE ALREADY DONE BY OTHER STUDENT")

In [ ]:
postgres.execute("""
    SELECT * FROM heroes
    LEFT JOIN heroes_items ON heroes.id = heroes_items.hero_id
    LEFT JOIN items ON heroes_items.item_id = items.id
    WHERE heroes.id = 33
""")
postgres.fetchone()

### MongoDB

In [ ]:
mongodb.heroes.update_one(
	{"name": "Visage"},
	{"$pull": {"items": {"name": "Blades_of_Attack"}}}
)
mongodb.heroes.find_one(
    {
    "name": "Visage"
    })

In [ ]:
mongodb.heroes.update_one(
	{"name": "Visage"},
	{"$set": {"name": "MEGA Blades of Attack"}}
)
mongodb.heroes.find_one({"name": "MEGA Blades of Attack"})

In [ ]:
mongodb.heroes.update_one(
	{"name": "Visage"},
	{"$push": {"items": mongodb.items.find_one({"name": "Visage"})}}
)
mongodb.heroes.find_one(
    {
    "name": "MEGA Blades of Attack"
    })

# Borrar elementos

### PostgreSQL

In [ ]:
# Borremos el elemento MEGA SWORD de la tabla items
try:
  postgres.execute(#Aquí tu código#
                   )
  conn.commit()
except:
  conn.rollback()
  print("DELETE ALREADY DONE BY OTHER STUDENT")

DELETE ALREADY DONE BY OTHER STUDENT


NOTA: Al fallar, lo ideal seria cambiar el constraint:


```
ALTER TABLE table_y   
    DROP CONSTRAINT constraint_name,   
    ADD CONSTRAINT constraint_name FOREIGN KEY (column_in_table_y)
          REFERENCES table_x (referenced_column_in_table_x) ON DELETE CASCADE;
```

### MongoDB

In [ ]:
mongodb.items.delete_one( {"name": "MEGA Blades of Attack"} )
mongodb.heroes.find_one(
    {
    "name": "Visage"
    })

In [ ]:
mongodb.heroes.update_one(
	{"name": "Visage"},
	{"$pull": {"items": {"name": "MEGA Blades of Attack"}}}
)
mongodb.heroes.find_one(
    {
    "name": "Visage"
    })

# Agrupar (GROUP BY / ORDER BY)

### PostgreSQL

In [ ]:
postgres.execute("""
SELECT type, count(*) FROM heroes GROUP BY type ORDER BY type DESC;
""")
hero_types = postgres.fetchall()

for hero in hero_types:
    pprint(hero)

### MongoDB

In [ ]:
aggregation = mongodb.heroes.aggregate([
    {"$group" : {"_id": "$type", "count": {"$sum": 1}}}
])
for results in aggregation:
    print(results)